In [ ]:
import pandas as pb



def taBort(frånFil, tillFil):
    datasmall = pb.read_excel(frånFil)

    missing_percentage = (datasmall.isnull().sum() / len(datasmall)) * 100

    cols_to_drop = missing_percentage[missing_percentage >= 99].index

    datasmall_filtered = datasmall.drop(columns=cols_to_drop)


    print("Shape after filtering:", datasmall_filtered.shape)

    datasmall_filtered.to_excel(tillFil, index=False)


taBort('LitenData.xlsx','LitenData_rensad.xlsx')
print(1)
taBort("LitenDataSorre.xlsx","LitenDataSorre_rensad.xlsx")
print(1)
taBort("LitenDataMyckeStorre.xlsx","LitenDataMyckeStorre_rensad.xlsx")
print(1)
taBort("updFAKSexp.xlsx","updFAKSexp_rensad.xlsx")



In [14]:
import pandas as pb
import torch
from transformers import PreTrainedTokenizerFast, pipeline, AutoTokenizer, AutoModelForCausalLM
from sklearn.model_selection import train_test_split


model_name = "AI-Sweden-Models/gpt-sw3-126m"
device = "cuda:0" if torch.cuda.is_available() else "cpu"
tokenizer = AutoTokenizer.from_pretrained(model_name, padding_side = 'left')
model = AutoModelForCausalLM.from_pretrained(model_name)
model.to(device)


datasmall = pb.read_excel('testtest.xlsx')

missing_percentage = (datasmall.isnull().sum() / len(datasmall)) * 100

cols_to_drop = missing_percentage[missing_percentage >= 97].index

datasmall = datasmall.drop(columns=cols_to_drop)


print("Shape after filtering:", datasmall.shape)



# tokaniserar en rad och kollar hur många tokens över.
# tokaniserar kolumn FY och tar bort bakifrån antal tokens vi vill ah bort tills den e tom
# tar all data från excel utom kolumn FY och tokaniserar men byter ut kolumn FY med det vi redan tokaniserat och tagit bort från
# sammanställer

# -----------------------------------------------------------------------------------------------


for i in range(datasmall.shape[0]):
    tokens = tokenizer.encode_plus(str(datasmall.iloc[i, -1]), return_tensors='pt')
    input_ids = tokens['input_ids'][0].tolist()
    tokens_remove_list = len(input_ids) - 2024
    if tokens_remove_list < 0:
        tokens_remove_list = 0

    # Remove tokens from the end until the length matches 2024
    while len(input_ids) > 2024:
        input_ids.pop()

    # Update the last column data in the dataframe with the decoded string
    datasmall.iloc[i, -1] = tokenizer.decode(input_ids)
    


# -----------------------------------------------------------------------------------------------

input_tokens = datasmall.iloc[:, 1:].values.tolist()  
output_tokens = datasmall.iloc[:, 0].values.tolist()

output_tokens = [str(value) for value in output_tokens]

processed_texts = []

for text in output_tokens:
    SokOrd = {'jmf','Jmf','JMF','jämfört','Jämfört','jämförelse','Jämförelse','tidigare','Tidigare','föregående','Föregående','2010','2011','2012','2013','2014','2015','2016','2017','2018','2019','2020','2021','2022','2023','2024'}

    for i in range(0, 3):
        for s in SokOrd:
            head, sep, tail = text.partition(s)
            if head == text:
                head = "" 
            txt = head[::-1]

            for tecken in txt:
                if tecken == '.':
                    head2, sep2, tail2 = tail.partition('.')
                    Second_head, Second_sep, Second_tail = txt.partition(tecken)
                    #Second_tail = Second_tail[1:] #ta bort sista mellanslag
                    new_txt = Second_tail[::-1]
                    text = new_txt + tail2
                    break
                
                if tecken == '(':
                    head2, sep2, tail2 = tail.partition(')')
                    Second_head, Second_sep, Second_tail = txt.partition(tecken)
                    Second_tail = Second_tail[1:] #ta bort sista mellanslag
                    new_txt = Second_tail[::-1]
                    text = new_txt + tail2
                    break

                if tecken == '?':
                    head2, sep2, tail2 = tail.partition('.')
                    Second_head, Second_sep, Second_tail = txt.partition(tecken)
                    Second_tail = Second_tail[1:] #ta bort sista mellanslag
                    new_txt = Second_tail[::-1]
                    text = new_txt + tail2
                    break



    processed_texts.append(text.strip()) 

output_tokens = processed_texts

par_tokens = [(output_tokens[i], input_tokens[i])for i in range(len(input_tokens))]


train_texts, val_and_test_texts = train_test_split(par_tokens, test_size=0.2, random_state=42) 
val_texts, test_texts = train_test_split(val_and_test_texts, test_size=0.5, random_state=42)

output_tokens_from_par_train_small = [str(item[0]) for item in train_texts]
output_tokens_from_par_val_small = [str(item[0]) for item in val_texts]
output_tokens_from_par_test_small = [str(item[0]) for item in test_texts]
input_tokens_from_par_train_small = [str(item[1]) for item in train_texts]
input_tokens_from_par_val_small = [str(item[1]) for item in val_texts]
input_tokens_from_par_test_small = [str(item[1]) for item in test_texts]

formatted_data_train = [f"<|endoftext|><s>User: Skriv en patientjornal efter en ultraljudsundersökning utifrån dessa värden: {input_token}<s>Bot:{output_token}<s>" 
                        for input_token, output_token in zip(input_tokens_from_par_train_small, output_tokens_from_par_train_small)]

formatted_data_val = [f"<|endoftext|><s>User: Skriv en patientjornal efter en ultraljudsundersökning utifrån dessa värden: {input_token}<s>Bot:{output_token}<s>" 
                      for input_token, output_token in zip(input_tokens_from_par_val_small, output_tokens_from_par_val_small)]

formatted_data_test = [f"<|endoftext|><s>User: Skriv en patientjornal efter en ultraljudsundersökning utifrån dessa värden: {input_token}<s>Bot:{output_token}<s>" 
                       for input_token, output_token in zip(input_tokens_from_par_test_small, output_tokens_from_par_test_small)]



class MyDataset(torch.utils.data.Dataset):
    def __init__(self, formatted_data, tokenizer):
        self.formatted_data = formatted_data
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.formatted_data)

    def __getitem__(self, idx):        
        formatted_data = self.formatted_data[idx]

        
        

        # Encode input sequence and labels using tokenizer
        inputs = self.tokenizer.encode_plus(
            formatted_data,
            return_tensors='pt',
            padding='max_length',
            #padding_side = 'left',
            #padding=True,
            #truncation=True,
            max_length = 1,       #har testat med bara truncation för att se om det är padding. och ja, det är padding som fuckar upp det för modellen.
            return_attention_mask=True,
            
        )


        return {
            "input_ids": inputs.input_ids.flatten(),
            "labels": inputs.input_ids.flatten(),
            "attention_mask": inputs.attention_mask.flatten(),
        }




AllData_train_small = MyDataset(formatted_data_train, tokenizer)
print(AllData_train_small.__getitem__(56))
AllData_val_small = MyDataset(formatted_data_val, tokenizer)
#random_val, AllData_val_small = train_test_split(AllData_val_small, test_size=0.02, random_state=42)
AllData_test_small = MyDataset(formatted_data_test, tokenizer)

max_input_length = max(len(example["labels"]) for example in AllData_train_small)

print("Maximum length of input_ids:", max_input_length)

Shape after filtering: (100, 241)
{'input_ids': tensor([    3,     2, 15088,  ...,  2265, 63443,     2]), 'labels': tensor([    3,     2, 15088,  ...,  2265, 63443,     2]), 'attention_mask': tensor([1, 1, 1,  ..., 1, 1, 1])}
Maximum length of input_ids: 1748


100


In [16]:
print(formatted_data_train[5])
print("-------------------------------------------------------------------------")
print(AllData_train_small[5]["input_ids"])
print("-------------------------------------------------------------------------")
print(tokenizer.encode_plus("Anamnes:"))
print(tokenizer.encode_plus("'"))

new_examples_list = []
target_number_start = "Anamnes:"
target_number_end = "'"
#target_number_start = str(target_number_start)
#target_number_end = str(target_number_end)
for example in formatted_data_train:
    example = tokenizer.encode_plus(example)['input_ids']
    #example = str(example)
    print("1ha")
    if(len(example) > 1750):
        # tokens
        antal_tokens_toRemove = len(example) - 1750
        print("toktorem: ", antal_tokens_toRemove)

        # string
        example = tokenizer.decode(example)
        head, sep, tail = example.partition(target_number_start)
        stage = sep + tail
        head2, sep2, tail2 = stage.partition(target_number_end)
        
        #tokens
        example = tokenizer.encode_plus(example)['input_ids']
        head = tokenizer.encode_plus(head)['input_ids']
        head2 = tokenizer.encode_plus(head2)['input_ids']
        tail2 = tokenizer.encode_plus(tail2)['input_ids']
        sep2 = tokenizer.encode_plus(sep2)['input_ids']
        print(len(head), " ", len(head2), " ", len(tail2), " ", len(sep2), " ", len(example))

        antal = len(head2)

        while len(head2) > (antal - antal_tokens_toRemove):
            print("len head2: ", len(head2))
            head2 = head2[:-1]
            print("en token borttagen")
            if len(head2) == 0:
                head2 = tokenizer.encode_plus("nan")['input_ids']
                print("inga token kvar")
                break

        example = head + head2 + sep2 + tail2
        print(len(head), " ", len(head2), " ", len(tail2), " ", len(sep2), " ", len(example))

    new_examples_list.append(tokenizer.decode(example))


formatted_data_train = (new_examples_list)


print(formatted_data_train[2])
print(len(formatted_data_train[2]))
print(len(tokenizer.encode_plus(formatted_data_train[0])['input_ids']))
print(len(str(tokenizer.encode_plus(formatted_data_train[0])['input_ids'])))
    


        <|endoftext|><s>        User: Skriv en patientjornal efter en ultraljudsundersökning utifrån dessa värden: ['101 - 153', 1.05, '0,26 - 0,90', 29, 3.0, 198.0, 'Flerstråligt', 'Normal', 'Bicuspid', 'Ja', nan, 35.0, '28 - 39', 16.1, 25.0, 11.5, 2.1, nan, '7,5 - 12,7', 25.0, '24 - 35', 11.5, 18.0, nan, 'Raphe mellan högra och vänstra kuspen. Kraftigt excentrisk, posteriort riktad jet. Vena contracta svår att beräkna. Spektraltät dopplerprofil. Slutdiastolisk hastighet i aorta desc 38 cm/s.', nan, '12 - 20', nan, '11 - 21', 14.0, '12,5 - 20,5', 12.0, '12 - 20', 10.0, '12 - 20', nan, nan, 'God', 'DS', 25.5, 2.17, nan, nan, nan, nan, 77.0, 'MB', 'EM', 171.0, '138 - 194', 1.3, '0,73 - 2,33', 1.32, '0,52 - 1,12', nan, '51 - 71', nan, '52 - 72', 59.0, '55 - 100\t45 - 54\t30 - 44\t< 30', nan, 'Gracil', nan, 'Gracil', nan, 55.0, 'Går själv', nan, 85.0, 'Man', 188.0, 17.0, 7.83, '6 - 13', nan, 'Normal', 25.0, '25 - 43', nan, nan, 20.0, '51 - 83', 14.0, 25.0, '0 - 20\t20 - 30\t30 - 40\t ≥ 40

In [13]:
print(tokenizer.encode_plus("nan"))
max_input_length = max(len(tokenizer.encode_plus(example)['input_ids']) for example in formatted_data_train)                                           
print("Maximum length of input_ids:", max_input_length)

{'input_ids': [19937], 'attention_mask': [1]}
Maximum length of input_ids: 1755


In [ ]:
print(datasmall.shape)
import torch

class MyDataset(torch.utils.data.Dataset):
    def __init__(self, formatted_data, tokenizer):
        self.formatted_data = formatted_data
        self.tokenizer = tokenizer
        self.max_label_length = 2048  # Maximum allowed length for labels

    def __len__(self):
        return len(self.formatted_data)

    def __getitem__(self, idx):        
        formatted_data = self.formatted_data[idx]

        # Encode input sequence and labels using tokenizer
        inputs = self.tokenizer.encode_plus(
            formatted_data,
            return_tensors='pt',
            padding='max_length',
            max_length=self.max_label_length,
            return_attention_mask=True,
        )

        label_length = len(inputs.input_ids.flatten())  # Get the length of labels

        return {
            "input_ids": inputs.input_ids.flatten(),
            "labels": inputs.input_ids.flatten(),
            "attention_mask": inputs.attention_mask.flatten(),
            "label_length": label_length  # Add the label length to the returned dictionary
        }

# Initialize your dataset
AllData_train_small = MyDataset(formatted_data_train, tokenizer)

# Count how many labels exceed 2048
count_exceeding_2048 = sum(1 for data in AllData_train_small if data["label_length"] > 2048)

print("Number of labels exceeding 2048:", count_exceeding_2048)

In [ ]:
#text = 'Max 6 mm perikardspatium med delvis organiserat innehåll utanför HK och ffa lateralt om VK (inget tidigare EKO tillgängligt för jämförelse, men regress ses vid översiktlig jmf med hjärt-MR 131220). Normalvid vänster kammare med väggtjocklek vid övre normalgräns, måttligt sänkt systolisk funktion (EF ca 40%). Lätt dilaterat vänster förmak. Normalstora högersidiga hjärtrum. Väs normal systolisk högerkammarfunktion. Lindrig mitralisinsuff. Maxsystoliskt PA-tryck ca 30 mmHg.'
#text = 'Normalstora hjärtrum. Normal vänster och högerkammarfunktion. EF 60%. Lindriga aorta, mitralis och tricuspidalisinsufficienser. Normalt PA-tryck. Jämfört med föregående us oförändrad bild.'
#text = 'Perikardvätska som omger hela hjärtat med max skikttjocklek c:a 15-20 mm inferiort och lateralt om vänster kammare; 10-15 mm utanför höger kammare samt omkring 30 mm utanför höger förmak med påverkan på dess vägg. Ingen tydlig påverkan på kammarnivå. Normalstor vänster kammare med normal systolisk funktion. Normalstor höger kammare med normal systolisk funktion. Normal klaffunktion. Således lindrig hemodynamisk påverkan av perikardvätskan men ingen ekokardiografisk tamponadbild. Ingen säkerställd förändring ses jämfört med undersökning från 2013-12-23. Inga bilder för jämförelse finns i EchoPac-servern från us. 5/1 som anges i remissen - var är den undersökningen utförd?'
text = 'Dilaterad höger kammare med nedsatt systolisk funktion. Vänster kammare med normal diameter, lätt septumhypertrofi samt lätt nedsatt global systolisk funktion (50-55%). Lätt septumavplaning. Normalvid v. cava inf, men andningsvariation kan ej bedömas. Systoliskt tryck i lungkretsloppet skattas måttligt förhöjt (minst 45-50 mmHg). Normalstora förmak. Lindriga insufficienser i samtliga klaffar. Jämfört med EKO 2011-10-20 ses nu lägre systolisk höger- och vänsterkammarfunktion. Nytillkommen lätt septumavplaning som kan tyda på nytillkommen PA-tryckstegring (PA-trycket kunde ej skattas 2011). Vid tidpunkten för svarsskrivning finns inga aktuella bilder arkiverade.'

import pandas as pd

SokOrd = {'jmf','Jmf','JMF','jämfört','Jämfört','jämförelse','Jämförelse','tidigare','Tidigare','föregående','Föregående','2010-','2011','2012-','2013-','2014-','2015-','2016-','2017-','2018-','2019-','2020-','2021-','2022-','2023-','2024-'}

for i in range(0, 3):
    for s in SokOrd:
        head, sep, tail = text.partition(s)
        if head == text:
            head = "" 
        txt = head[::-1]

        for tecken in txt:
            if tecken == '.':
                head2, sep2, tail2 = tail.partition('.')
                Second_head, Second_sep, Second_tail = txt.partition(tecken)
                #Second_tail = Second_tail[1:] #ta bort sista mellanslag
                new_txt = Second_tail[::-1]
                text = new_txt + tail2
                break
            
            if tecken == '(':
                head2, sep2, tail2 = tail.partition(')')
                Second_head, Second_sep, Second_tail = txt.partition(tecken)
                Second_tail = Second_tail[1:] #ta bort sista mellanslag
                new_txt = Second_tail[::-1]
                text = new_txt + tail2
                break

            if tecken == '?':
                head2, sep2, tail2 = tail.partition('.')
                Second_head, Second_sep, Second_tail = txt.partition(tecken)
                Second_tail = Second_tail[1:] #ta bort sista mellanslag
                new_txt = Second_tail[::-1]
                text = new_txt + tail2
                break



print(text.strip())




In [ ]:
text = 'Max 6 mm perikardspatium med delvis organiserat innehåll utanför HK och ffa lateralt om VK (inget tidigare EKO tillgängligt för jämförelse, men regress ses vid översiktlig jmf med hjärt-MR 131220). Normalvid vänster kammare med väggtjocklek vid övre normalgräns, måttligt sänkt systolisk funktion (EF ca 40%). Lätt dilaterat vänster förmak. Normalstora högersidiga hjärtrum. Väs normal systolisk högerkammarfunktion. Lindrig mitralisinsuff. Maxsystoliskt PA-tryck ca 30 mmHg.'
import pandas as pd

for tecken in text:
    if tecken == 'jmf':
        head, sep, tail = text.partition('jmf')
        head_inv= head[::-1]
        for tckn in head_inv:
            if tckn == '(' or '.':
                head2, sep2, tail2 = head_inv.partition('(')
                if sep2 == '(':
                head2, sep2, tail2 = head_inv.partition('.')
                break

In [ ]:
import pandas as pd
datasmall = pd.read_excel('LitenData.xlsx')

output_tokens = datasmall.iloc[:, 0].values.tolist()
processed_texts = []

for text in output_tokens:
    SokOrd = {'jmf','Jmf','JMF','jämfört','Jämfört','jämförelse','Jämförelse','tidigare','Tidigare','föregående','Föregående','2010','2011','2012','2013','2014','2015','2016','2017','2018','2019','2020','2021','2022','2023','2024'}

    for i in range(0, 3):
        for s in SokOrd:
            head, sep, tail = text.partition(s)
            if head == text:
                head = "" 
            txt = head[::-1]

            for tecken in txt:
                if tecken == '.':
                    head2, sep2, tail2 = tail.partition('.')
                    Second_head, Second_sep, Second_tail = txt.partition(tecken)
                    #Second_tail = Second_tail[1:] #ta bort sista mellanslag
                    new_txt = Second_tail[::-1]
                    text = new_txt + tail2
                    break
                
                if tecken == '(':
                    head2, sep2, tail2 = tail.partition(')')
                    Second_head, Second_sep, Second_tail = txt.partition(tecken)
                    Second_tail = Second_tail[1:] #ta bort sista mellanslag
                    new_txt = Second_tail[::-1]
                    text = new_txt + tail2
                    break

                if tecken == '?':
                    head2, sep2, tail2 = tail.partition('.')
                    Second_head, Second_sep, Second_tail = txt.partition(tecken)
                    Second_tail = Second_tail[1:] #ta bort sista mellanslag
                    new_txt = Second_tail[::-1]
                    text = new_txt + tail2
                    break



    processed_texts.append(text.strip())  # Append the processed text to the list, stripping any leading/trailing whitespace

# Convert the list to a DataFrame
df = pd.DataFrame(processed_texts, columns=['Processed Text'])

# Write the DataFrame to an Excel file
df.to_excel('processed_texts.xlsx', index=False)

In [ ]:
print(Second_sep)

In [ ]:
print(output_tokens[85])
print("----------------------------")
print(processed_texts[85])

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
datasmall = pd.read_excel('updFAKSexp_rensad.xlsx')
#print(datasmall.shape)


'''
input_tokens = [(input_text) for input_text in datasmall.iloc[:, 1:].iterrows()]
print(len(input_tokens))
output_tokens = [(output_text) for output_text in datasmall.iloc[:, 0]]
'''

input_tokens = datasmall.iloc[:, 1:].values.tolist()  
output_tokens = datasmall.iloc[:, 0].values.tolist()
output_tokens = [str(value) for value in output_tokens]

processed_texts = []

for text in output_tokens:
    SokOrd = {'jmf','Jmf','JMF','jämfört','Jämfört','jämförelse','Jämförelse','tidigare','Tidigare','föregående','Föregående','2010','2011','2012','2013','2014','2015','2016','2017','2018','2019','2020','2021','2022','2023','2024'}

    for i in range(0, 3):
        for s in SokOrd:
            head, sep, tail = text.partition(s)
            if head == text:
                head = "" 
            txt = head[::-1]

            for tecken in txt:
                if tecken == '.':
                    head2, sep2, tail2 = tail.partition('.')
                    Second_head, Second_sep, Second_tail = txt.partition(tecken)
                    #Second_tail = Second_tail[1:] #ta bort sista mellanslag
                    new_txt = Second_tail[::-1]
                    text = new_txt + tail2
                    break
                
                if tecken == '(':
                    head2, sep2, tail2 = tail.partition(')')
                    Second_head, Second_sep, Second_tail = txt.partition(tecken)
                    Second_tail = Second_tail[1:] #ta bort sista mellanslag
                    new_txt = Second_tail[::-1]
                    text = new_txt + tail2
                    break

                if tecken == '?':
                    head2, sep2, tail2 = tail.partition('.')
                    Second_head, Second_sep, Second_tail = txt.partition(tecken)
                    Second_tail = Second_tail[1:] #ta bort sista mellanslag
                    new_txt = Second_tail[::-1]
                    text = new_txt + tail2
                    break



    processed_texts.append(text.strip()) 



df = pd.DataFrame(processed_texts, columns=['Processed Text'])

# Write the DataFrame to an Excel file
df.to_excel('processed_texts.xlsx', index=False)

output_tokens = processed_texts


par_tokens = [(output_tokens[i], input_tokens[i])for i in range(len(input_tokens))]

train_texts, val_and_test_texts = train_test_split(par_tokens, test_size=0.2, random_state=42) 
val_texts, test_texts = train_test_split(val_and_test_texts, test_size=0.5, random_state=42)

output_tokens_from_par_train_small = [str(item[0]) for item in train_texts]
output_tokens_from_par_val_small = [str(item[0]) for item in val_texts]
output_tokens_from_par_test_small = [str(item[0]) for item in test_texts]
input_tokens_from_par_train_small = [str(item[1]) for item in train_texts]
input_tokens_from_par_val_small = [str(item[1]) for item in val_texts]
input_tokens_from_par_test_small = [str(item[1]) for item in test_texts]
